In [2]:
import cv2 as cv
import glob
import pickle

from matplotlib import pyplot as plt
from os import path

%matplotlib inline

In [3]:
print('Using OpenCV version', cv.__version__)

Using OpenCV version 3.1.0


In [4]:
with open('data/labels.pickle', 'rb') as handle:
  data = pickle.load(handle)

In [41]:
def get_img_data(fidx, imgidx):
    img_data = {}
    img_data['name'] = data['img_names'][fidx][imgidx][0]
    img_data['blur'] = data['blur_labels'][fidx][imgidx][0]
    img_data['expression'] = data['expression_labels'][fidx][imgidx][0]
    img_data['illumination'] = data['illumination_labels'][fidx][imgidx][0]
    img_data['occlusion'] = data['occlusion_labels'][fidx][imgidx][0]
    img_data['invalid'] = data['invalid_labels'][fidx][imgidx][0]
    img_data['pose'] = data['pose_labels'][fidx][imgidx][0]
    
    return img_data

In [42]:
print(get_img_data(5, 5))

{'pose': 0, 'name': '1', 'occlusion': 0, 'expression': 0, 'invalid': 0, 'blur': 1, 'illumination': 0}


In [ ]:
# Get and normalize file paths.
img_paths = [path.normpath(p) for p in glob.iglob('data/WIDER/test/**/*.jpg', recursive=True)]

face_cascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')

print(len(img_paths))

cv_dict = {}
for img_path in img_paths[:300]:
    img = cv.imread(img_path)
    grayscale_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    detected_faces = face_cascade.detectMultiScale(grayscale_img, 1.3, 5)
    
    img_name = path.basename(img_path)
    cv_dict[img_name[:-4]] = len(detected_faces)

In [ ]:
#Test performance of OpenCV
hints = 0;
missed = 0;

for fidx, folder in enumerate(data['img_names']):
    for idx, img_name in enumerate(folder):
        if get_img_data(fidx, idx)['occlusion'] == 0 and \
           get_img_data(fidx, idx)['blur'] == 0 and \
           get_img_data(fidx, idx)['pose'] == 0 and \
           get_img_data(fidx, idx)['illumination'] == 0:
            if img_name in cv_dict.keys():
                target_detected_faces = len(data['face_rectangles'][fidx][idx])
                predicted_detected_faces = cv_dict[img_name]
                hints = hints + min(target_detected_faces, predicted_detected_faces)
                missed = missed + abs(target_detected_faces - predicted_detected_faces)        

In [ ]:
print(hints)
print(missed)